# Декораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys
from datetime import datetime
from typing import Callable
from functools import wraps

In [2]:
sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [3]:
output = open("./some_test_data.txt", 'w')

In [4]:
output.write('123')

3

In [5]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdout` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [6]:
original_write = sys.stdout.write
new_line_flag = True


def my_write(string_text: str) -> int:
    global new_line_flag
    now = f'[{datetime.now():%Y-%m-%d %H:%M}]: '
    if new_line_flag:
        string_text = f'{now}{string_text}'
        new_line_flag = False
    if string_text == '\n':
        new_line_flag = True
    else:
        string_text = string_text.replace('\n', f'\n{now}')
    return original_write(string_text)


sys.stdout.write = my_write

In [7]:
print('1, 2, 3')

[2022-11-23 09:35]: 1, 2, 3


Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

In [8]:
# ещё тесты
print(1, 2, 3)
print('1, 2, 3,\n4, 5, 6')
print(1, 2, 3, 4, 5, 6, sep='\n')

[2022-11-23 09:35]: 1 2 3
[2022-11-23 09:35]: 1, 2, 3,
[2022-11-23 09:35]: 4, 5, 6
[2022-11-23 09:35]: 1
[2022-11-23 09:35]: 2
[2022-11-23 09:35]: 3
[2022-11-23 09:35]: 4
[2022-11-23 09:35]: 5
[2022-11-23 09:35]: 6


In [9]:
sys.stdout.write = original_write

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [10]:
def timed_output(function: Callable) -> Callable:
    """декоратор, который добавляет в каждую строку вывода временную мету"""
    original_write = sys.stdout.write
    new_line_flag = True

    def my_write(string_text: str):
        nonlocal new_line_flag
        now = f'[{datetime.now():%Y-%m-%d %H:%M}]: '
        if new_line_flag:
            string_text = f'{now}{string_text}'
            new_line_flag = False
        if string_text == '\n':
            new_line_flag = True
        else:
            string_text = string_text.replace('\n', f'\n{now}')
        return original_write(string_text)

    def wrapper(name: str):
        sys.stdout.write = my_write
        return_value = function(name)
        sys.stdout.write = original_write
        return return_value

    return wrapper

In [11]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [12]:
print_greeting("Nikita")

[2022-11-23 09:35]: Hello, Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

In [13]:
@timed_output
def print_tests(name: str):
    """ещё тесты"""
    print(f'Hello, {name}!')
    print('')
    print(1, 2, 3)
    print('1, 2, 3,\n4, 5, 6')
    print(1, 2, 3, 4, 5, 6, sep='\n')

In [14]:
print_tests("Nikita")

[2022-11-23 09:35]: Hello, Nikita!
[2022-11-23 09:35]: 
[2022-11-23 09:35]: 1 2 3
[2022-11-23 09:35]: 1, 2, 3,
[2022-11-23 09:35]: 4, 5, 6
[2022-11-23 09:35]: 1
[2022-11-23 09:35]: 2
[2022-11-23 09:35]: 3
[2022-11-23 09:35]: 4
[2022-11-23 09:35]: 5
[2022-11-23 09:35]: 6


## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [15]:
def redirect_output(filepath: str, *file_args, **file_kwargs) -> Callable:
    """генерирует декоратор, перенаправляющий вывод из stdout в файл filepath"""
    def decorator(function: Callable) -> Callable:
        @wraps(function)
        def wrapper(*args, **kwargs):
            original_stdout = sys.stdout
            with open(filepath, 'w', *file_args, **file_kwargs) as otf:
                sys.stdout = otf
                return_value = function(*args, **kwargs)
            sys.stdout = original_stdout
            return return_value
        return wrapper
    return decorator

In [16]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [17]:
calculate()

In [18]:
# вывод файла в консоль windows
!type function_output.txt

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 


In [19]:
calculate.__name__

'calculate'